In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/kdsh2024/btc_15m.csv')

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['datetime'], df['close'], label='Close Price')
plt.xlabel('date')
plt.ylabel('close')
plt.show()

In [ ]:
df.set_index('datetime', inplace=True)

In [ ]:
df_resampled_hr = df.resample('1H').last()
df_resampled_hr = df_resampled_hr.ffill()
df_resampled_hr.head()

In [ ]:
df_resampled_dl = df.resample('1D').last()
df_resampled_dl = df_resampled_dl.ffill()
df_resampled_dl.head()

In [ ]:
!pip install ta

In [ ]:
import pandas as pd
import ta
import glob

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = df_resampled_hr['close'].values

In [ ]:
mean, std = data.mean(), data.std()
data = (data - mean) / std

In [ ]:
data_tensor = torch.FloatTensor(data).view(-1, 1).to(device)

In [ ]:
def generate_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return torch.stack(sequences), torch.stack(targets)

In [ ]:
sequence_length = 24

In [ ]:
sequences, targets = generate_sequences(data_tensor, sequence_length)

In [ ]:
train_size = int(len(sequences) * 0.8)
train_data, test_data = sequences[:train_size], sequences[train_size:]
train_targets, test_targets = targets[:train_size], targets[train_size:]
train_data, train_targets = train_data.to(device), train_targets.to(device)
test_data, test_targets = test_data.to(device), test_targets.to(device)

In [ ]:
batch_size = 256
train_dataset = TensorDataset(train_data, train_targets)
test_dataset = TensorDataset(test_data, test_targets)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
input_size = 1
hidden_size = 256
output_size = 1
num_layers = 2

In [ ]:
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)
import os
output_dir = '/kaggle/working/'
model_path = os.path.join(output_dir, 'model.pth')
torch.save(model.state_dict(), model_path)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
optimizer.zero_grad()

In [ ]:
num_epochs = 100

In [ ]:
lossi = []
for epoch in range(num_epochs):
    for batch_seq, batch_target in train_loader:
        optimizer.zero_grad()
        output = model(batch_seq)
        loss = criterion(output, batch_target)
        loss.backward()
        optimizer.step()

    lossi.append(loss.item())
    if(epoch%10==0):
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [ ]:
plt.plot(lossi)

In [ ]:
model.eval()
with torch.no_grad():
    test_predictions = model(test_data)

In [ ]:
test_predictions = (test_predictions * std) + mean
test_targets = (test_targets * std) + mean

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(test_targets.cpu().numpy(), test_predictions.cpu().numpy())
rmse = mse**0.5
print(f'Mean Squared Error on Test: {mse:.4f}')
print(f' Root Mean Squared Error on Test: {rmse:.4f}')

In [ ]:
test_predictions = (test_predictions * std) + mean
test_targets = (test_targets * std) + mean

plt.figure(figsize=(12, 6))
plt.plot(test_targets.cpu().numpy(), label='Actual Close Prices', color='blue')
plt.plot(test_predictions.cpu().numpy(), label='Predicted Close Prices', color='red')
plt.xlabel('Time Steps')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [ ]:
test_data.size()

In [ ]:
test_data_cpu = test_data.cpu()
test_data_cpu = test_data_cpu.detach().numpy()
test_data_cpu = test_data_cpu.reshape(test_data.size(0), test_data.size(1))

In [ ]:
num_rows, num_columns = test_data_cpu.shape
print(f"Number of rows: {num_rows}, Number of columns: {num_columns}")

In [ ]:


# Convert the CPU tensor to NumPy array and create a DataFrame
df = pd.DataFrame(test_data_cpu)

In [ ]:
df.head()

In [ ]:
df.drop(df.columns[0:24], axis=1, inplace=True)

In [ ]:
df['actual_close'] = test_targets.cpu()
df['predicted_close'] = test_predictions.cpu()

In [ ]:
!apt-get install -y libta-lib0 libta-lib-dev


In [ ]:
!pip install TA-Lib

In [ ]:
pip install ta TA-lib


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame 'df' with columns like 'actual_close', 'predicted_close'

# Define initial capital (starting amount of money)
initial_capital = 10000

# Bollinger Bands parameters
bb_period = 20
bb_std_dev = 2

# Moving Average parameters
ma_short_window = 50
ma_long_window = 200

# Calculate Bollinger Bands without talib
df['rolling_mean'] = df['actual_close'].rolling(window=bb_period).mean()
df['rolling_std'] = df['actual_close'].rolling(window=bb_period).std()
df['bb_upper'] = df['rolling_mean'] + (bb_std_dev * df['rolling_std'])
df['bb_lower'] = df['rolling_mean'] - (bb_std_dev * df['rolling_std'])

# Calculate Moving Averages
df['ma_short'] = df['actual_close'].rolling(window=ma_short_window, min_periods=1).mean()
df['ma_long'] = df['actual_close'].rolling(window=ma_long_window, min_periods=1).mean()

# Initialize position and portfolio value
position = 1  # 1 for long (buy), -1 for short (sell), 0 for hold
portfolio_value = initial_capital

# Adaptive stop-loss parameters
stop_loss_percent = 0.25  # Initial stop loss percentage

# Lists to store trading signals, portfolio values, and stop loss levels
signals = []
portfolio_values = []
stop_loss_levels = []
net_profit_values = []

# Iterate through the DataFrame and apply the strategy
for i in range(1, len(df)):
    # Bollinger Bands Strategy
    if df['actual_close'].iloc[i] < df['bb_lower'].iloc[i - 1]:
        position = 0  # Sell signal
    elif df['actual_close'].iloc[i] > df['bb_upper'].iloc[i - 1]:
        position = 1  # Buy signal

    # Update adaptive stop-loss level
    stop_loss_level = df['actual_close'].iloc[i] * (1 - stop_loss_percent)
    stop_loss_levels.append(stop_loss_level)

    # Check if the position should be closed based on stop-loss
    if position == 1 and df['actual_close'].iloc[i] < stop_loss_level:
        position = 0  # Close long position

    # Update the position and calculate the portfolio value
    signals.append(position)
    portfolio_value = portfolio_value * (1 + position * df['actual_close'].pct_change().iloc[i])
    portfolio_values.append(portfolio_value)
    
    net_profit = portfolio_value - initial_capital
    net_profit_values.append(net_profit)

# Add signals, portfolio values, and stop-loss levels to the DataFrame
df['signal'] = [0] + signals  # 0 for the initial day
df['portfolio_value'] = [initial_capital] + portfolio_values
df['stop_loss'] = [0] + stop_loss_levels  # Stop-loss level for each day
df['net_profit'] = [0] + net_profit_values


# Plot the Bollinger Bands, Moving Averages, and trading signals
plt.figure(figsize=(12, 8))
plt.plot(df.index, df['actual_close'], label='Actual Close', color='blue')
plt.plot(df.index, df['bb_upper'], label='Upper Bollinger Band', color='orange')
plt.plot(df.index, df['rolling_mean'], label='Middle Bollinger Band', color='black')
plt.plot(df.index, df['bb_lower'], label='Lower Bollinger Band', color='orange')
plt.plot(df.index, df['ma_short'], label=f'{ma_short_window}-day Short MA', color='green')
plt.plot(df.index, df['ma_long'], label=f'{ma_long_window}-day Long MA', color='red')
plt.scatter(df.index[df['signal'] == 1], df['actual_close'][df['signal'] == 1], marker='^', color='green', label='Buy Signal')
plt.scatter(df.index[df['signal'] == 0], df['actual_close'][df['signal'] == 0], marker='v', color='red', label='Sell Signal')
plt.plot(df.index, df['stop_loss'], label='Adaptive Stop Loss', linestyle='--')

# Plot the portfolio value over time
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['portfolio_value'], label='Portfolio Value', color='purple')
plt.title('Adjusted Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Investment')
plt.legend()
plt.show()

# Plot the net profit over time
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['net_profit'], label='Net Profit', color='orange')
plt.title('Net Profit Over Time')
plt.xlabel('Date')
plt.ylabel('Net Profit ($)')
plt.axhline(y=0, color='r', linestyle='--', label='Breakeven')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame 'df' with columns like 'actual_close', 'predicted_close'


# Lists to store trade history
buy_signals = df.index[df['signal'] == 1].tolist()
sell_signals = df.index[df['signal'] == 0].tolist()

# Create subplots
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1, figsize=(12, 18), sharex=True)

# Plot Bollinger Bands

ax1.plot(df.index, df['bb_upper'], label='Upper Bollinger Band', color='orange')
ax1.plot(df.index, df['rolling_mean'], label='Middle Bollinger Band', color='black')
ax1.plot(df.index, df['bb_lower'], label='Lower Bollinger Band', color='blue')
# ax1.scatter(buy_signals, df['actual_close'][df['signal'] == 1], marker='^', color='green', label='Buy Signal')
# ax1.scatter(sell_signals, df['actual_close'][df['signal'] == 0], marker='v', color='red', label='Sell Signal')
ax1.legend()

# Plot Moving Averages
# ax2.plot(df.index, df['actual_close'], label='Actual Close', color='blue')
ax2.plot(df.index, df['ma_short'], label=f'{ma_short_window}-day Short MA', color='green')
ax2.plot(df.index, df['ma_long'], label=f'{ma_long_window}-day Long MA', color='red')
ax2.legend()

# Plot Adaptive Stop Loss
ax3.plot(df.index, df['stop_loss'], label='Adaptive Stop Loss', linestyle='--', color='purple')
ax3.scatter(buy_signals, df['stop_loss'][df['signal'] == 1], marker='^', color='green', label='Buy Signal')
ax3.scatter(sell_signals, df['stop_loss'][df['signal'] == 0], marker='v', color='red', label='Sell Signal')
ax3.legend()

# Plot Buy and Sell Signals
ax4.plot(df.index, df['signal'], label='Buy/Sell Signal', color='blue')
ax4.legend()

# Plot Actual vs Predicted Close
ax5.plot(df.index, df['actual_close'], label='Actual Close', color='blue')
ax5.plot(df.index, df['predicted_close'], label='Predicted Close', color='orange')
ax5.legend()

# Plot Trade History vs Time
ax6.scatter(df.index[df['signal'] == 1], [1] * len(buy_signals), marker='^', color='green', label='Buy Signal')
ax6.scatter(df.index[df['signal'] == 0], [-1] * len(sell_signals), marker='v', color='red', label='Sell Signal')
ax6.set_yticks([])
ax6.legend()

# Calculate Daily Returns
df['daily_returns'] = df['portfolio_value'].pct_change()

# Calculate Cumulative Returns
df['cumulative_returns'] = (1 + df['daily_returns']).cumprod()

# Plot Equity Curve
fig, ax7 = plt.subplots(figsize=(12, 8))

ax7.plot(df.index, df['portfolio_value'], label='Equity Curve', color='purple')
ax7.set_ylabel('Equity Value ($)')
ax7.legend()

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame 'df' with columns like 'actual_close', 'predicted_close', 'signal', 'portfolio_value'

# Define initial capital (starting amount of money)
initial_capital = 10000

# ... (your previous code for strategy implementation)

# Calculate net profit
df['net_profit'] = df['portfolio_value'] - initial_capital

# Calculate benchmark return (e.g., buy and hold strategy)
df['benchmark_return'] = df['actual_close'] / df['actual_close'].iloc[0] * initial_capital

# Calculate net profit exceeding benchmark return
net_profit_exceeding_benchmark = df['net_profit'].iloc[-1] - (df['benchmark_return'].iloc[-1] - initial_capital)
print(f'Net Profit Exceeding Benchmark Return: ${net_profit_exceeding_benchmark:.2f}')

# Calculate net profit exceeding benchmark return as a percentage
net_profit_percentage = (net_profit_exceeding_benchmark / initial_capital) * 100
print(f'Net Profit Exceeding Benchmark Return: {net_profit_percentage:.2f}%')

# Calculate the benchmark return as a percentage
benchmark_return_percentage = (df['benchmark_return'].iloc[-1] - initial_capital) / initial_capital * 100



# Calculate the duration of each trade as a percentage
df['trade_duration'] = (df['signal'] != df['signal'].shift(1)).cumsum()
trade_durations = df.groupby('trade_duration')['actual_close'].count()
max_duration_percentage = (trade_durations.max() / len(df)) * 100
print(f'Max Duration Time of Single Trade: {max_duration_percentage:.2f}% of total trading days')

# Calculate the duration of each trade
df['trade_duration'] = (df['signal'] != df['signal'].shift(1)).cumsum()
max_duration_time = df.groupby('trade_duration')['actual_close'].count().max()
print(f'Max Duration Time of Single Trade: {max_duration_time} days')

In [ ]:
# ... (your previous code)



In [ ]:
! pip install pandas_market_calendars


In [ ]:

beginning_portfolio_value = 10000
ending_portfolio_value = df['portfolio_value'].iloc[-1]

# Calculate the number of years

investment_period_years = 1029 / 252  # Assuming 252 trading days in a year

# Calculate annualized return
annualized_return = ((ending_portfolio_value / beginning_portfolio_value) ** (1 / investment_period_years)) - 1

print(f"Annualized Return: {annualized_return:.2%}")

In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd

# Define the stablecoin ticker symbol
stablecoin_ticker = 'USDC-USD'

# Download historical data
stablecoin_data = yf.download(stablecoin_ticker, start='2018-01-01', end='2022-01-31')['Adj Close']

# Calculate daily returns
stablecoin_returns = stablecoin_data.pct_change()

# Assume annual compounding for simplicity
risk_free_rate = (1 + stablecoin_returns.mean()) ** 252 - 1

print(f"Risk-Free Rate (proxy using stablecoin): {risk_free_rate:.4%}")


In [ ]:
import numpy as np



# Calculate Maximum Drawdown
peak_index = np.argmax(df['portfolio_value'])
trough_index = np.argmin(df['portfolio_value'][:peak_index + 1])
max_drawdown = (df['portfolio_value'].iloc[peak_index] - df['portfolio_value'].iloc[trough_index]) / df['portfolio_value'].iloc[peak_index]

print(f"Maximum Drawdown: {max_drawdown:.2%}")

# Calculate Sharpe Ratio

portfolio_return = annualized_return  
portfolio_std_dev = df['daily_returns'].std() * np.sqrt(252)  # Assuming 252 trading days in a year

sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

In [ ]:
import numpy as np

# Calculate the net profit over time
df['net_profit'] = df['portfolio_value'] - initial_capital

# Identify the peak and trough of the net profit
peak_value = df['net_profit'].max()
trough_value = df['net_profit'].min()

# Calculate potential rewards and risks
potential_rewards = peak_value - initial_capital
potential_risks = initial_capital - trough_value

# Display the potential rewards and risks
print(f'Potential Rewards: {potential_rewards:.2f}')
print(f'Potential Risks: {potential_risks:.2f}')

# Calculate the risk-reward ratio
risk_reward_ratio = potential_rewards / potential_risks

# Display the risk-reward ratio
print(f'Risk-Reward Ratio: {risk_reward_ratio:.2f}')


In [ ]:
import pandas as pd
import numpy as np

# Calculate Total Closed Trades
total_closed_trades = df['signal'].sum()
print(f'Total Closed Trades: {total_closed_trades}')

In [ ]:


# Assuming 'df' is the DataFrame containing the trading signals, portfolio values, etc.

# Calculate Daily Returns
df['daily_returns'] = df['portfolio_value'].pct_change()

# Calculate Cumulative Returns
df['cumulative_returns'] = (1 + df['daily_returns']).cumprod()

# Calculate Trades
df['buy_signals'] = np.where(df['signal'] == 1, 1, 0)
df['sell_signals'] = np.where(df['signal'] == 0, 1, 0)

# Calculate Gross Profit and Gross Loss
df['gross_profit'] = np.where(df['buy_signals'] == 1, df['actual_close'].pct_change(), 0)
df['gross_loss'] = np.where(df['sell_signals'] == 1, -df['actual_close'].pct_change(), 0)

# Calculate Net Profit
df['net_profit'] = df['portfolio_value'] - df['portfolio_value'].shift(1)



# Calculate Win Rate
winning_trades = df['gross_profit'].gt(0).sum()
win_rate = winning_trades / total_closed_trades if total_closed_trades > 0 else 0


# Calculate Average Winning and Losing Trades
average_winning_trade = df[df['gross_profit'] > 0]['gross_profit'].mean()
average_losing_trade = df[df['gross_loss'] > 0]['gross_loss'].mean()

# Calculate Buy and Hold Return of BTC
buy_and_hold_return = (df['actual_close'].iloc[-1] / df['actual_close'].iloc[0]) - 1

# Calculate Largest Winning and Losing Trades
largest_winning_trade = df['gross_profit'].max()
largest_losing_trade = df['gross_loss'].min()



# Calculate Average Holding Duration per Trade
df['holding_duration'] = df.groupby((df['signal'] != df['signal'].shift()).cumsum())['signal'].cumcount() + 1
average_holding_duration = df.groupby('signal')['holding_duration'].mean().mean()


# Display the results
print(f'Gross Profit: {df["gross_profit"].sum()}')
print(f'Net Profit: {df["net_profit"].sum()}')

print(f'Win Rate: {win_rate * 100:.2f}%')
print(f'Gross Loss: {df["gross_loss"].sum()}')
print(f'Average Winning Trade: {average_winning_trade}')
print(f'Average Losing Trade: {average_losing_trade}')
print(f'Buy and Hold Return of BTC: {buy_and_hold_return * 100:.2f}%')
print(f'Largest Winning Trade: {largest_winning_trade}')
print(f'Largest Losing Trade: {largest_losing_trade}')

print(f'Average Holding Duration per Trade: {average_holding_duration:.2f} periods')


In [ ]:
# Calculate Drawdown
df['peak_value'] = df['portfolio_value'].cummax()
df['drawdown'] = (df['peak_value'] - df['portfolio_value']) / df['peak_value']

# Calculate Max Drawdown and Average Drawdown in Running Trades
max_drawdown = df['drawdown'].max()
average_drawdown = df[df['signal'] == 1]['drawdown'].mean()

print(f'Max Drawdown in Running Trades: {max_drawdown * 100:.2f}%')
print(f'Average Drawdown in Running Trades: {average_drawdown * 100:.2f}%')



In [ ]:
# Calculate Sortino Ratio
downside_returns = df[df['daily_returns'] < 0]['daily_returns']
sortino_ratio = (df['daily_returns'].mean() / downside_returns.std()) * np.sqrt(252)
print(f'Sortino Ratio: {sortino_ratio}')

In [ ]:
buy_and_hold_return = (df['actual_close'].iloc[-1] / df['actual_close'].iloc[0]) - 1

# Display the results
print(f'Buy and Hold Return of BTC: {buy_and_hold_return * 100:.2f}%')


# **Backtesting over 1 Jan 2015 to 31 Dec 2017**

In [ ]:
import pandas as pd
import torch

# Load your dataset
df = pd.read_csv('/kaggle/input/backtesting-2015-to-2017/BTC-USD.csv')

# Assuming your dataset has a 'Close' column and 'Date' column
df.set_index('Date', inplace=True)

# Drop the 'Adj Close' column if it exists
if 'Adj Close' in df.columns:
    df = df.drop(columns=['Adj Close'])

# Remove the row containing column names
df = df.iloc[1:]

# Normalize the 'Close' column
data = df['Close'].values
mean, std = data.mean(), data.std()
data = (data - mean) / std

# Convert data to PyTorch tensor
data_tensor = torch.FloatTensor(data).view(-1, 1)

# Function to generate sequences and targets
def generate_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return torch.stack(sequences), torch.stack(targets)

# Define sequence length
sequence_length = 24

# Generate sequences and targets
sequences, targets = generate_sequences(data_tensor, sequence_length)

print(targets.size())
print(sequences.size())
# Use the trained LSTM model
model = LSTMModel(input_size, hidden_size, output_size, num_layers).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Move sequences to the device
sequences = sequences.to(device)
# Make predictions with the trained model
with torch.no_grad():
    predictions = model(sequences[-len(df)-sequence_length+1:])  # Adjust the sequences for prediction

# Convert predictions back to the original scale
predictions = (predictions * std) + mean

# Create the 'actual_close' column in the DataFrame
df['actual_close'] = df['Close']
print(predictions.cpu().numpy())

# Create the 'predicted_close' column in the DataFrame
df['predicted_close'] = [None]*(sequence_length) + predictions.cpu().numpy().tolist()[-len(df):]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a DataFrame 'df' with columns like 'actual_close', 'predicted_close'

# Define initial capital (starting amount of money)
initial_capital = 10000

# Bollinger Bands parameters
bb_period = 20
bb_std_dev = 2

# Moving Average parameters
ma_short_window = 50
ma_long_window = 200

# Calculate Bollinger Bands without talib
df['rolling_mean'] = df['actual_close'].rolling(window=bb_period).mean()
df['rolling_std'] = df['actual_close'].rolling(window=bb_period).std()
df['bb_upper'] = df['rolling_mean'] + (bb_std_dev * df['rolling_std'])
df['bb_lower'] = df['rolling_mean'] - (bb_std_dev * df['rolling_std'])

# Calculate Moving Averages
df['ma_short'] = df['actual_close'].rolling(window=ma_short_window, min_periods=1).mean()
df['ma_long'] = df['actual_close'].rolling(window=ma_long_window, min_periods=1).mean()

# Initialize position and portfolio value
position = 1  # 1 for long (buy), -1 for short (sell), 0 for hold
portfolio_value = initial_capital

# Lists to store trading signals, portfolio values, and stop loss levels
signals = []
portfolio_values = []
stop_loss_levels = []

# Iterate through the DataFrame and apply the strategy
for i in range(1, len(df)):
    # Bollinger Bands Strategy
    if df['actual_close'].iloc[i] < df['bb_lower'].iloc[i - 1]:
        position = 0  # Sell signal
    elif df['actual_close'].iloc[i] > df['bb_upper'].iloc[i - 1]:
        position = 1  # Buy signal

    # Update adaptive stop-loss level
    stop_loss_level = df['actual_close'].iloc[i] * (1 - stop_loss_percent)
    stop_loss_levels.append(stop_loss_level)

    # Check if the position should be closed based on stop-loss
    if position == 1 and df['actual_close'].iloc[i] < stop_loss_level:
        position = 0  # Close long position

    # Update the position and calculate the portfolio value
    signals.append(position)
    portfolio_value = portfolio_value * (1 + position * df['actual_close'].pct_change().iloc[i])
    portfolio_values.append(portfolio_value)

# Add signals, portfolio values, and stop-loss levels to the DataFrame
df['signal'] = [0] + signals  # 0 for the initial day
df['portfolio_value'] = [initial_capital] + portfolio_values
df['stop_loss'] = [0] + stop_loss_levels  # Stop-loss level for each day

# Display relevant information
print(f'Total Closed Trades: {df["signal"].sum()}')
print(f'Maximum Drawdown: {max_drawdown * 100:.2f}%')
print(f'Sharpe Ratio: {sharpe_ratio:.4f}')

# Plot results
# (You may need to adapt the plotting code depending on your specific needs)


In [ ]:
# Calculate the net profit over time
df['net_profit'] = df['portfolio_value'] - initial_capital

# Identify the peak and trough of the net profit
peak_value = df['net_profit'].max()
trough_value = df['net_profit'].min()

# Calculate potential rewards and risks
potential_rewards = peak_value - initial_capital
potential_risks = initial_capital - trough_value

# Display the potential rewards and risks
print(f'Potential Rewards: {potential_rewards:.2f}')
print(f'Potential Risks: {potential_risks:.2f}')

# Calculate the risk-reward ratio
risk_reward_ratio = potential_rewards / potential_risks

# Display the risk-reward ratio
print(f'Risk-Reward Ratio: {risk_reward_ratio:.2f}')

In [ ]:

# Plot the portfolio value over time
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['portfolio_value'], label='Portfolio Value', color='purple')
plt.title('Adjusted Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Investment')
plt.legend()
plt.show()

# Calculate the net profit column
df['net_profit'] = df['portfolio_value'] - initial_capital

# Plot the net profit over time
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['net_profit'], label='Net Profit', color='orange')
plt.title('Net Profit Over Time')
plt.xlabel('Date')
plt.ylabel('Net Profit ($)')
plt.axhline(y=0, color='r', linestyle='--', label='Breakeven')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a DataFrame 'df' with columns like 'actual_close', 'predicted_close'


# Lists to store trade history
buy_signals = df.index[df['signal'] == 1].tolist()
sell_signals = df.index[df['signal'] == 0].tolist()

# Create subplots
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(12, 18), sharex=True)

# Plot Bollinger Bands

ax1.plot(df.index, df['bb_upper'], label='Upper Bollinger Band', color='orange')
ax1.plot(df.index, df['rolling_mean'], label='Middle Bollinger Band', color='black')
ax1.plot(df.index, df['bb_lower'], label='Lower Bollinger Band', color='blue')
# ax1.scatter(buy_signals, df['actual_close'][df['signal'] == 1], marker='^', color='green', label='Buy Signal')
# ax1.scatter(sell_signals, df['actual_close'][df['signal'] == 0], marker='v', color='red', label='Sell Signal')
ax1.legend()

# Plot Moving Averages
# ax2.plot(df.index, df['actual_close'], label='Actual Close', color='blue')
ax2.plot(df.index, df['ma_short'], label=f'{ma_short_window}-day Short MA', color='green')
ax2.plot(df.index, df['ma_long'], label=f'{ma_long_window}-day Long MA', color='red')
ax2.legend()

# Plot Adaptive Stop Loss
ax3.plot(df.index, df['stop_loss'], label='Adaptive Stop Loss', linestyle='--', color='purple')
ax3.scatter(buy_signals, df['stop_loss'][df['signal'] == 1], marker='^', color='green', label='Buy Signal')
ax3.scatter(sell_signals, df['stop_loss'][df['signal'] == 0], marker='v', color='red', label='Sell Signal')
ax3.legend()

# Plot Buy and Sell Signals
ax4.plot(df.index, df['signal'], label='Buy/Sell Signal', color='blue')
ax4.legend()




# Plot Trade History vs Time
ax5.scatter(df.index[df['signal'] == 1], [1] * len(buy_signals), marker='^', color='green', label='Buy Signal')
ax5.scatter(df.index[df['signal'] == 0], [-1] * len(sell_signals), marker='v', color='red', label='Sell Signal')
ax5.set_yticks([])
ax5.legend()

# Calculate Daily Returns
df['daily_returns'] = df['portfolio_value'].pct_change()

# Calculate Cumulative Returns
df['cumulative_returns'] = (1 + df['daily_returns']).cumprod()

# Plot Equity Curve
fig, ax7 = plt.subplots(figsize=(12, 8))

ax7.plot(df.index, df['portfolio_value'], label='Equity Curve', color='purple')
ax7.set_ylabel('Equity Value ($)')
ax7.legend()

plt.tight_layout()
plt.show()





In [ ]:


# Assuming 'df' is the DataFrame containing the trading signals, portfolio values, etc.

# Calculate Daily Returns
df['daily_returns'] = df['portfolio_value'].pct_change()

# Calculate Cumulative Returns
df['cumulative_returns'] = (1 + df['daily_returns']).cumprod()

# Calculate Trades
df['buy_signals'] = np.where(df['signal'] == 1, 1, 0)
df['sell_signals'] = np.where(df['signal'] == 0, 1, 0)

# Calculate Gross Profit and Gross Loss
df['gross_profit'] = np.where(df['buy_signals'] == 1, df['actual_close'].pct_change(), 0)
df['gross_loss'] = np.where(df['sell_signals'] == 1, -df['actual_close'].pct_change(), 0)

# Calculate Net Profit
df['net_profit'] = df['portfolio_value'] - df['portfolio_value'].shift(1)



# Calculate Win Rate
winning_trades = df['gross_profit'].gt(0).sum()
win_rate = winning_trades / total_closed_trades if total_closed_trades > 0 else 0


# Calculate Average Winning and Losing Trades
average_winning_trade = df[df['gross_profit'] > 0]['gross_profit'].mean()
average_losing_trade = df[df['gross_loss'] > 0]['gross_loss'].mean()

# Calculate Buy and Hold Return of BTC
buy_and_hold_return = (df['actual_close'].iloc[-1] / df['actual_close'].iloc[0]) - 1

# Calculate Largest Winning and Losing Trades
largest_winning_trade = df['gross_profit'].max()
largest_losing_trade = df['gross_loss'].min()



# Calculate Average Holding Duration per Trade
df['holding_duration'] = df.groupby((df['signal'] != df['signal'].shift()).cumsum())['signal'].cumcount() + 1
average_holding_duration = df.groupby('signal')['holding_duration'].mean().mean()


# Display the results
print(f'Gross Profit: {df["gross_profit"].sum()}')
print(f'Net Profit: {df["net_profit"].sum()}')

print(f'Win Rate: {win_rate * 100:.2f}%')
print(f'Gross Loss: {df["gross_loss"].sum()}')
print(f'Average Winning Trade: {average_winning_trade}')
print(f'Average Losing Trade: {average_losing_trade}')
print(f'Buy and Hold Return of BTC: {buy_and_hold_return * 100:.2f}%')
print(f'Largest Winning Trade: {largest_winning_trade}')
print(f'Largest Losing Trade: {largest_losing_trade}')

print(f'Average Holding Duration per Trade: {average_holding_duration:.2f} periods')


In [ ]:
import numpy as np

# Calculate the net profit over time
df['net_profit'] = df['portfolio_value'] - initial_capital

# Identify the peak and trough of the net profit
peak_value = df['net_profit'].max()
trough_value = df['net_profit'].min()

# Calculate potential rewards and risks
potential_rewards = peak_value - initial_capital
potential_risks = initial_capital - trough_value

# Display the potential rewards and risks
print(f'Potential Rewards: {potential_rewards:.2f}')
print(f'Potential Risks: {potential_risks:.2f}')

# Calculate the risk-reward ratio
risk_reward_ratio = potential_rewards / potential_risks

# Display the risk-reward ratio
print(f'Risk-Reward Ratio: {risk_reward_ratio:.2f}')


In [ ]:
# Calculate Sortino Ratio
downside_returns = df[df['daily_returns'] < 0]['daily_returns']
sortino_ratio = (df['daily_returns'].mean() / downside_returns.std()) * np.sqrt(252)
print(f'Sortino Ratio: {sortino_ratio}')

In [ ]:
buy_and_hold_return = (df['actual_close'].iloc[-1] / df['actual_close'].iloc[0]) - 1

# Display the results
print(f'Buy and Hold Return of BTC: {buy_and_hold_return * 100:.2f}%')